![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Use pretrained `explain_document` Pipeline

### Stages

 * DocumentAssembler
 * SentenceDetector
 * Tokenizer
 * Lemmatizer
 * Stemmer
 * Part of Speech
 * SpellChecker (Norvig)


In [12]:
import sys
sys.path.append('../../')

#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher

### Let's create a Spark Session for our app

In [13]:
spark = SparkSession.builder \
    .appName("Explain_Document")\
    .master("local[*]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jars", "/tmp/sparknlp.jar")\
    .config("spark.driver.extraClassPath", "/tmp/sparknlp.jar")\
    .config("spark.executor.extraClassPath", "/tmp/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

#### This is our testing document, we'll use it to exemplify all different pipeline stages.

In [14]:
testDoc = [
"French author who helped pioner the science-fiction genre. \
Verne wrate about space, air, and underwater travel before \
navigable aircrast and practical submarines were invented, \
and before any means of space travel had been devised. "
]

In [15]:
pipeline = PipelineModel.load("../demo_pipelines/explain_document_ml_en_1.8.0_2.4_1552689539570")

#### We are not interested in handling big datasets, let's switch to LightPipelines for speed.

In [16]:
from sparknlp.base import LightPipeline
lp = LightPipeline(pipeline)
result = lp.annotate(testDoc)

#### Let's analyze these results - first let's see what sentences we detected

In [17]:
[content['sentence'] for content in result]

[['French author who helped pioner the science-fiction genre.',
  'Verne wrate about space, air, and underwater travel before navigable aircrast and practical submarines were invented, and before any means of space travel had been devised.']]

#### Now let's see how those sentences were tokenized

In [18]:
[content['token'] for content in result]


[['French',
  'author',
  'who',
  'helped',
  'pioner',
  'the',
  'science-fiction',
  'genre',
  '.',
  'Verne',
  'wrate',
  'about',
  'space',
  ',',
  'air',
  ',',
  'and',
  'underwater',
  'travel',
  'before',
  'navigable',
  'aircrast',
  'and',
  'practical',
  'submarines',
  'were',
  'invented',
  ',',
  'and',
  'before',
  'any',
  'means',
  'of',
  'space',
  'travel',
  'had',
  'been',
  'devised',
  '.']]

#### Notice some spelling errors? the pipeline takes care of that as well

In [19]:
[content['spell'] for content in result]

[['French',
  'author',
  'who',
  'helped',
  'pioneer',
  'the',
  'science-fiction',
  'genre',
  '.',
  'Verne',
  'wrote',
  'about',
  'space',
  ',',
  'air',
  ',',
  'and',
  'underwater',
  'travel',
  'before',
  'navigable',
  'aircraft',
  'and',
  'practical',
  'submarines',
  'were',
  'invented',
  ',',
  'and',
  'before',
  'any',
  'means',
  'of',
  'space',
  'travel',
  'had',
  'been',
  'devised',
  '.']]

#### Now let's see the lemmas

In [20]:
[content['lemmas'] for content in result]

[['French',
  'author',
  'who',
  'help',
  'pioneer',
  'the',
  'science-fiction',
  'genre',
  '.',
  'Verne',
  'write',
  'about',
  'space',
  ',',
  'air',
  ',',
  'and',
  'underwater',
  'travel',
  'before',
  'navigable',
  'aircraft',
  'and',
  'practical',
  'submarine',
  'be',
  'invent',
  ',',
  'and',
  'before',
  'any',
  'mean',
  'of',
  'space',
  'travel',
  'have',
  'be',
  'devise',
  '.']]

#### Let's check the stems, any difference with the lemmas shown bebore?

In [21]:
[content['stems'] for content in result]

[['french',
  'author',
  'who',
  'help',
  'pioneer',
  'the',
  'science-fict',
  'genr',
  '.',
  'vern',
  'wrote',
  'about',
  'space',
  ',',
  'air',
  ',',
  'and',
  'underwat',
  'travel',
  'befor',
  'navig',
  'aircraft',
  'and',
  'practic',
  'submarin',
  'were',
  'invent',
  ',',
  'and',
  'befor',
  'ani',
  'mean',
  'of',
  'space',
  'travel',
  'had',
  'been',
  'devis',
  '.']]

#### Now it's the turn of Part Of Speech(POS)

In [22]:
pos = [content['pos'] for content in result]
token = [content['token'] for content in result]
# let's put token and tag together
list(zip(token[0], pos[0]))

[('French', 'JJ'),
 ('author', 'NN'),
 ('who', 'WP'),
 ('helped', 'VBD'),
 ('pioner', 'NN'),
 ('the', 'DT'),
 ('science-fiction', 'NN'),
 ('genre', 'NN'),
 ('.', '.'),
 ('Verne', 'NNP'),
 ('wrate', 'VBD'),
 ('about', 'IN'),
 ('space', 'NN'),
 (',', ','),
 ('air', 'NN'),
 (',', ','),
 ('and', 'CC'),
 ('underwater', 'JJ'),
 ('travel', 'NN'),
 ('before', 'IN'),
 ('navigable', 'JJ'),
 ('aircrast', 'NN'),
 ('and', 'CC'),
 ('practical', 'JJ'),
 ('submarines', 'NNS'),
 ('were', 'VBD'),
 ('invented', 'VBN'),
 (',', ','),
 ('and', 'CC'),
 ('before', 'IN'),
 ('any', 'DT'),
 ('means', 'NNS'),
 ('of', 'IN'),
 ('space', 'NN'),
 ('travel', 'NN'),
 ('had', 'VBD'),
 ('been', 'VBN'),
 ('devised', 'VBN'),
 ('.', '.')]